In [1]:
import os
import sys

In [2]:
print(os.getcwd())

/labs/gevaertlab/users/yyhhli/code/vae/EDA


In [6]:
sys.path.insert(1, "/labs/gevaertlab/users/yyhhli/code/vae/")

In [7]:
from torch.utils.data.dataloader import DataLoader
from applications.application import Application
from datasets.utils import sitk2tensor
from datasets import PATCH_DATASETS

[03-03 15:32:56 | utils:_init_num_threads] Note: NumExpr detected 40 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[03-03 15:32:56 | utils:_init_num_threads] NumExpr defaulting to 8 threads.


In [8]:

stfrg_train_patch = PATCH_DATASETS["StanfordRadiogenomicsPatchAugDataset"](root_dir=None,
                                                                            transform=sitk2tensor,
                                                                            split='train')
stfrg_train_patch_dataloader = DataLoader(dataset=stfrg_train_patch,
                                            batch_size=1,
                                            shuffle=False,
                                            drop_last=False,
                                            num_workers=4,
                                            pin_memory=True)
stfrg_test_patch = PATCH_DATASETS["StanfordRadiogenomicsPatchDataset"](root_dir=None,
                                                                        transform=sitk2tensor,
                                                                        split='test')
stfrg_test_patch_dataloader = DataLoader(dataset=stfrg_test_patch,
                                            batch_size=1,
                                            shuffle=False,
                                            drop_last=False,
                                            num_workers=4,
                                            pin_memory=True)



[03-03 15:33:05 | patch_stanfordradiogenomics:StanfordRadiogenomicsPatchAugDataset] patient split: train:100, test:43
[03-03 15:33:05 | patch_stanfordradiogenomics:StanfordRadiogenomicsPatchDataset] patient split: train:100, test:43


In [ ]:

for task in ["StfAJCC", "StfHisGrade", "StfNStage", "StfTStage", "StfLymphInvasion", "StfEGFRMutation", "StfKRASMutation"]:  # "StfVolume", "StfLymphInvasion"
    for version in [49, 51, 53, 57, 58, 59, 60]:
        print(
            f"======= Predicting {task} with model version {version} =======")
        app = Application(log_name='VAE3D32AUG',
                            version=version,
                            task_name=task,
                            task_kwds={"task_type": "classification"},
                            base_model_name='VAE3D',
                            dataloaders={'train': stfrg_train_patch_dataloader,
                                        'val': stfrg_test_patch_dataloader})

        result_dict, pred_dict, pred_stats, hparam_dict = app.task_prediction(tune_hparams=True, models='all')


In [10]:
task = "StfAJCC"
version = 49
print(
            f"======= Predicting {task} with model version {version} =======")
app = Application(log_name='VAE3D32AUG',
                    version=version,
                    task_name=task,
                    task_kwds={"task_type": "classification"},
                    base_model_name='VAE3D',
                    dataloaders={'train': stfrg_train_patch_dataloader,
                                'val': stfrg_test_patch_dataloader})

result_dict, hparam_dict = app.task_prediction(tune_hparams=True, models='random_forest', bootstrapping=True)


======= Predicting StfAJCC with model version 49 =======


[03-03 15:33:34 | export:  Exporter] initializing embeddings
[03-03 15:33:35 | application:Application] -----prediction for task StfAJCC-----
[03-03 15:33:35 | application:Application] Loading best hparams ...
[03-03 15:33:35 | models:predict_task] Before transform: X shape = train:(100, 4096), val:(43, 4096); Y shape = train:(100,), val:(43,)
[03-03 15:33:35 | models:data_summary] X shape = train:(97, 4096), val:(42, 4096); Y shape = train:(97,), val:(42,)
Y classes = train: 
  value count
0     I    58
1   II+    39; val: 
  value count
0     I    29
1   II+    13
[03-03 15:33:35 | models:predict_with_model] ======random_forest======


 | 1.0 secs.
initializing | 2.0 secs.
['random_forest']


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices